In [1]:
## import nba_api to collect information
from nba_api.stats import endpoints
from nba_api.stats.static import teams 
teams = teams.get_teams()

## other pacakges used
import pandas as pd

In [2]:
team_list = ['1610612747','1610612738','1610612741']

In [3]:
team_data = pd.DataFrame()

for x in team_list:
    data = endpoints.teamyearbyyearstats.TeamYearByYearStats(team_id=x)
    df = data.get_data_frames()[0]
    
    if len(team_data.columns) == 0:
        team_data = df
    else:
        team_data = team_data.append(df)

In [5]:
#only use selected columns from data set
team_data = team_data[['TEAM_ID','TEAM_NAME','NBA_FINALS_APPEARANCE','FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','PF','STL','TOV','BLK']]

#clean data set so that only data is used when FGA information is captured
team_data = team_data[team_data.FGA > 0]

#create column taht indicates in team won championship or not. 1 = champions
team_data['CHAMPION'] = team_data['NBA_FINALS_APPEARANCE'].apply(lambda x: 1 if x == 'LEAGUE CHAMPION' else 0)

#drop NBA_FINALS_APPEARANCE column as not used
team_data.drop(columns=['NBA_FINALS_APPEARANCE'], inplace=True)

In [6]:
team_data

,TEAM_ID,TEAM_NAME,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,PF,STL,TOV,BLK,CHAMPION
34,1610612747,Lakers,3964,7512,0.528,10,96,0.104,1495,2031,0.736,1235,2433,3668,2519,1931,844,1584,479,0
35,1610612747,Lakers,3854,7250,0.532,58,226,0.257,1712,2272,0.754,1095,2499,3594,2455,2054,726,1578,478,0
36,1610612747,Lakers,3952,7254,0.545,90,295,0.305,1702,2232,0.763,1063,2550,3613,2575,1931,695,1537,481,1
37,1610612747,Lakers,3834,7343,0.522,138,409,0.337,1812,2329,0.778,1101,2555,3656,2433,2031,693,1467,419,0
38,1610612747,Lakers,3740,7245,0.516,164,447,0.367,2012,2550,0.789,1127,2515,3642,2428,1853,728,1358,482,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,1610612741,Bulls,3169,7141,0.444,623,1831,0.340,1474,1848,0.798,999,2797,3796,1851,1455,643,1112,393,0
51,1610612741,Bulls,3170,7285,0.435,906,2549,0.355,1194,1574,0.759,790,2872,3662,1923,1571,626,1147,289,0
52,1610612741,Bulls,3266,7205,0.453,745,2123,0.351,1328,1695,0.783,718,2799,3517,1796,1663,603,1159,351,0
53,1610612741,Bulls,2573,5762,0.447,793,2282,0.348,1006,1332,0.755,680,2041,2721,1510,1417,652,1005,265,0


In [55]:
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pydotplus
from IPython.display import Image
from sklearn.preprocessing import LabelEncoder
from imblearn.ensemble import BalancedRandomForestClassifier

In [34]:
le = LabelEncoder()

In [37]:
le.fit(team_data["TEAM_NAME"])
team_data["TEAM_NAME"] = le.transform(team_data["TEAM_NAME"])

In [38]:
team_data.head()

,TEAM_ID,TEAM_NAME,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,PF,STL,TOV,BLK,CHAMPION
34,1610612747,2,3964,7512,0.528,10,96,0.104,1495,2031,0.736,1235,2433,3668,2519,1931,844,1584,479,0
35,1610612747,2,3854,7250,0.532,58,226,0.257,1712,2272,0.754,1095,2499,3594,2455,2054,726,1578,478,0
36,1610612747,2,3952,7254,0.545,90,295,0.305,1702,2232,0.763,1063,2550,3613,2575,1931,695,1537,481,1
37,1610612747,2,3834,7343,0.522,138,409,0.337,1812,2329,0.778,1101,2555,3656,2433,2031,693,1467,419,0
38,1610612747,2,3740,7245,0.516,164,447,0.367,2012,2550,0.789,1127,2515,3642,2428,1853,728,1358,482,1


In [69]:
target = ["CHAMPION"]
y = team_data.loc[:, target].copy()

X = team_data.drop(columns="CHAMPION")
X = pd.get_dummies(X)

In [70]:
X.describe()

,TEAM_ID,TEAM_NAME,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,PF,STL,TOV,BLK
count,1.170000e+02,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.00000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000,117.000000
mean,1.610613e+09,1.000000,3153.888889,6755.589744,0.465966,429.623932,1226.170940,0.33547,1535.769231,2032.230769,0.755504,982.692308,2480.888889,3463.581197,1921.769231,1758.367521,650.059829,1223.641026,406.076923
std,3.757751e+00,0.820008,390.872582,614.184687,0.027996,253.316543,689.790577,0.04779,240.774518,305.311073,0.032683,168.820475,247.167882,308.961141,274.040187,223.615803,95.031781,156.608697,70.024030
min,1.610613e+09,0.000000,1539.000000,3837.000000,0.401000,10.000000,96.000000,0.10400,745.000000,1077.000000,0.679000,509.000000,1394.000000,1967.000000,1017.000000,1109.000000,389.000000,754.000000,169.000000
25%,1.610613e+09,0.000000,2996.000000,6592.000000,0.446000,226.000000,659.000000,0.32800,1410.000000,1908.000000,0.737000,891.000000,2400.000000,3365.000000,1782.000000,1663.000000,602.000000,1141.000000,361.000000
50%,1.610613e+09,1.000000,3144.000000,6880.000000,0.463000,446.000000,1265.000000,0.35100,1536.000000,2052.000000,0.756000,999.000000,2495.000000,3528.000000,1888.000000,1776.000000,645.000000,1223.000000,406.000000
75%,1.610613e+09,2.000000,3434.000000,7141.000000,0.482000,575.000000,1599.000000,0.36400,1670.000000,2248.000000,0.778000,1095.000000,2612.000000,3635.000000,2091.000000,1902.000000,712.000000,1305.000000,454.000000
max,1.610613e+09,2.000000,3964.000000,7547.000000,0.545000,1032.000000,2829.000000,0.40300,2012.000000,2743.000000,0.832000,1290.000000,2985.000000,3855.000000,2575.000000,2223.000000,987.000000,1743.000000,575.000000


In [71]:
y['CHAMPION'].value_counts()

0    99
1    18
Name: CHAMPION, dtype: int64

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [73]:
scaler = StandardScaler()

In [74]:
X_scaler = scaler.fit(X_train)

In [75]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [76]:
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train_scaled, y_train)

<ipython-input-76-03a5d3e36309>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  brf.fit(X_train_scaled, y_train)


BalancedRandomForestClassifier(random_state=1)

In [77]:
y_pred = brf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.7222222222222222

In [78]:
confusion_matrix(y_test, y_pred)

array([[21,  6],
       [ 1,  2]], dtype=int64)

In [79]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.95      0.78      0.67      0.86      0.72      0.52        27
          1       0.25      0.67      0.78      0.36      0.72      0.51         3

avg / total       0.88      0.77      0.68      0.81      0.72      0.52        30



In [80]:
importances = brf.feature_importances_ 
indices = np.argsort(importances)[::-1]
for f in range(X.shape[1]):
    print(f"{X.columns.values[indices[f]]}: ({importances[indices[f]]})")

FG_PCT: (0.12945105789764993)
REB: (0.0929102605898803)
PF: (0.06448342592779531)
STL: (0.061770453194847065)
DREB: (0.05683775733547118)
AST: (0.05658183308232468)
OREB: (0.05655028437294177)
TOV: (0.05419597946277716)
FG3A: (0.05343184674875873)
FGA: (0.05003067272068436)
FTA: (0.04936231997095674)
FT_PCT: (0.04570919877739274)
FTM: (0.0456831165009753)
BLK: (0.04468918461822785)
FG3_PCT: (0.0436895711241134)
FGM: (0.041111337414598284)
FG3M: (0.0391576121187849)
TEAM_ID: (0.00824185924369748)
TEAM_NAME: (0.006112228898122862)
